In [ ]:
import math
from dataclasses import dataclass

@dataclass
class WeatherConditions:
    oat: float  # Fahrenheit
    humidity: float  # % RH
    wet_bulb: float  # Fahrenheit
    dew_point: float  # Fahrenheit
    enthalpy: float  # BTU/lb
    pressure: float = 29.92  # inHg

def calculate_psychrometrics(t_f: float, rh: float, pressure_inhg: float = 29.92) -> WeatherConditions:
    """
    Calculate psychrometric properties based on Temp (F) and RH (%).
    """
    # Convert T to Celsius for standard formulas
    t_c = (t_f - 32) * 5/9
    
    # Saturation Vapor Pressure (hPa) - Magnus formula
    es = 6.112 * math.exp((17.67 * t_c) / (t_c + 243.5))
    
    # Actual Vapor Pressure (hPa)
    e = es * (rh / 100.0)
    
    # Dew Point (Celsius)
    # Inverse Magnus
    # ln(e/6.112) = (17.67 * Tdp) / (Tdp + 243.5)
    # let alpha = ln(e/6.112)
    # alpha * (Tdp + 243.5) = 17.67 * Tdp
    # alpha * Tdp + alpha * 243.5 = 17.67 * Tdp
    # Tdp * (17.67 - alpha) = alpha * 243.5
    # Tdp = (alpha * 243.5) / (17.67 - alpha)
    
    try:
        alpha = math.log(e / 6.112)
        t_dp_c = (alpha * 243.5) / (17.67 - alpha)
        t_dp_f = (t_dp_c * 9/5) + 32
    except ValueError:
        t_dp_f = t_f # Low humidity edge case
        
    # Humidity Ratio (W) - lb H2O / lb dry air
    # P_atm in hPa
    p_atm_hpa = pressure_inhg * 33.8639
    
    # W = 0.622 * e / (P - e)
    w = 0.622 * e / (p_atm_hpa - e)
    
    # Enthalpy (BTU/lb)
    # h = 0.24 * T_f + W * (1061 + 0.444 * T_f)
    h = 0.24 * t_f + w * (1061 + 0.444 * t_f)
    
    # Wet Bulb (Stull's approximation)
    # T_wb = T * arctan(0.151977 * (RH + 8.313659)^(1/2)) + arctan(T + RH) - arctan(RH - 1.676331) + 0.00391838 * (RH)^(3/2) * arctan(0.023101 * RH) - 4.686035
    # Note: T in Celsius, RH in %
    
    term1 = t_c * math.atan(0.151977 * math.sqrt(rh + 8.313659))
    term2 = math.atan(t_c + rh)
    term3 = math.atan(rh - 1.676331)
    term4 = 0.00391838 * (rh**1.5) * math.atan(0.023101 * rh)
    t_wb_c = term1 + term2 - term3 + term4 - 4.686035
    t_wb_f = (t_wb_c * 9/5) + 32
    
    return WeatherConditions(
        oat=t_f,
        humidity=rh,
        wet_bulb=t_wb_f,
        dew_point=t_dp_f,
        enthalpy=h,
        pressure=pressure_inhg
    )

# Test
cond = calculate_psychrometrics(95, 40)
print(f"95F, 40% RH -> WB: {cond.wet_bulb:.1f}, DP: {cond.dew_point:.1f}, H: {cond.enthalpy:.1f}")
